In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
da_2010_dec = pd.read_excel('Data/rpt.00013060.0000000000000000.DAMLZHBSPP_2010.xlsx', sheet_name='Dec_1')
rt_2010_dec = pd.read_excel('Data/rpt.00013061.0000000000000000.RTMLZHBSPP_2010.xlsx', sheet_name='Dec_1')

In [3]:
zone = "LZ_HOUSTON"
dam_z = da_2010_dec[da_2010_dec["Settlement Point"] == zone].copy()
rtm_z = rt_2010_dec[rt_2010_dec["Settlement Point Name"] == zone].copy()

In [4]:
dam_z

,Delivery Date,Hour Ending,Repeated Hour Flag,Settlement Point,Settlement Point Price
8,12/01/2010,01:00,N,LZ_HOUSTON,35.36
22,12/01/2010,02:00,N,LZ_HOUSTON,30.45
36,12/01/2010,03:00,N,LZ_HOUSTON,29.35
50,12/01/2010,04:00,N,LZ_HOUSTON,29.41
64,12/01/2010,05:00,N,LZ_HOUSTON,30.72
...,...,...,...,...,...
10354,12/31/2010,20:00,N,LZ_HOUSTON,40.35
10368,12/31/2010,21:00,N,LZ_HOUSTON,35.30
10382,12/31/2010,22:00,N,LZ_HOUSTON,30.21
10396,12/31/2010,23:00,N,LZ_HOUSTON,27.08


In [5]:
rtm_z

,Delivery Date,Delivery Hour,Delivery Interval,Repeated Hour Flag,Settlement Point Name,Settlement Point Type,Settlement Point Price
32,12/01/2010,1,1,N,LZ_HOUSTON,LZ,25.08
33,12/01/2010,1,2,N,LZ_HOUSTON,LZ,23.20
34,12/01/2010,1,3,N,LZ_HOUSTON,LZ,23.45
35,12/01/2010,1,4,N,LZ_HOUSTON,LZ,22.17
88,12/01/2010,2,1,N,LZ_HOUSTON,LZ,21.96
...,...,...,...,...,...,...,...
41587,12/31/2010,23,4,N,LZ_HOUSTON,LZ,22.16
41640,12/31/2010,24,1,N,LZ_HOUSTON,LZ,26.41
41641,12/31/2010,24,2,N,LZ_HOUSTON,LZ,25.58
41642,12/31/2010,24,3,N,LZ_HOUSTON,LZ,25.80


In [6]:
dam_z["HE_clean"] = dam_z["Hour Ending"].str.replace("24:00", "00:00")

dam_z["datetime"] = pd.to_datetime(
    dam_z["Delivery Date"] + " " + dam_z["HE_clean"],
    format="%m/%d/%Y %H:%M"
)
mask_24 = dam_z["Hour Ending"] == "24:00"
dam_z.loc[mask_24, "datetime"] += pd.Timedelta(days=1)

dam_z = dam_z.set_index("datetime").sort_index()
dam_z = dam_z.rename(columns={"Settlement Point Price": "DAM"})
dam_hourly = dam_z[["DAM"]]


mask_24 = rtm_z["Delivery Hour"] == 24
rtm_z.loc[mask_24, "Delivery Hour"] = 0
rtm_z.loc[mask_24, "Delivery Date"] = (
    pd.to_datetime(rtm_z.loc[mask_24, "Delivery Date"]) + pd.Timedelta(days=1)
).dt.strftime("%m/%d/%Y")

rtm_z["datetime"] = pd.to_datetime(
    rtm_z["Delivery Date"] + " " + rtm_z["Delivery Hour"].astype(str),
    format="%m/%d/%Y %H"
)

rtm_z = rtm_z.set_index("datetime").sort_index()
rtm_z = rtm_z.rename(columns={"Settlement Point Price": "RTM_15min"})

rtm_hourly = rtm_z["RTM_15min"].groupby("datetime").mean().to_frame("RTM")

In [7]:
dam_hourly

,DAM
datetime,
2010-12-01 01:00:00,35.36
2010-12-01 02:00:00,30.45
2010-12-01 03:00:00,29.35
2010-12-01 04:00:00,29.41
2010-12-01 05:00:00,30.72
...,...
2010-12-31 20:00:00,40.35
2010-12-31 21:00:00,35.30
2010-12-31 22:00:00,30.21


In [8]:
rtm_hourly

,RTM
datetime,
2010-12-01 01:00:00,23.4750
2010-12-01 02:00:00,21.4675
2010-12-01 03:00:00,21.3100
2010-12-01 04:00:00,22.3500
2010-12-01 05:00:00,24.3925
...,...
2010-12-31 20:00:00,31.1775
2010-12-31 21:00:00,27.2700
2010-12-31 22:00:00,25.1375


In [9]:
# federal funds rate in december of 12/1/2010 is 0.18%
fed_funds_df = pd.read_csv('Data/FEDFUNDS.csv')
fed_funds_df

,observation_date,FEDFUNDS
0,10/1/2010,0.19
1,11/1/2010,0.19
2,12/1/2010,0.18


In [10]:
fed_funds_rate = fed_funds_df[fed_funds_df['observation_date'] == '12/1/2010']['FEDFUNDS'].values[0]
fed_funds_rate

0.18

In [11]:
import netCDF4

temp = netCDF4.Dataset('Data/data_stream-oper_stepType-instant.nc')



In [12]:
lat = np.asarray(temp.variables['latitude'])
lon = np.asarray(temp.variables['longitude'])
# temperature 2 meters above ground level
t2m = np.asarray(temp.variables['t2m']) # shape (time, lat, lon)

In [13]:
# time is in hours (31 * 24 = 744 hours)
t2m.shape 

(744, 721, 1440)

In [14]:
# taking the sub grid of interest (around Texas)
temp_sub = t2m[:, 208:257, 1006:1064]
temp_sub.shape

(744, 49, 58)

In [15]:
import yfinance as yf

data = yf.download(
    "SPY",
    start="2010-12-01",
    end="2011-01-01",
    interval="1d"
)
data


/tmp/ipykernel_338171/190695544.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SPY,SPY,SPY,SPY,SPY
Date,,,,,
2010-12-01,92.360092,92.535635,91.734233,91.741861,221037200
2010-12-02,93.543129,93.611824,92.451690,92.505117,191213600
2010-12-03,93.794998,93.901852,93.199669,93.222566,151288900
2010-12-06,93.695793,93.909500,93.497348,93.596568,103050500
2010-12-07,93.749237,94.649865,93.695810,94.596438,206581000
2010-12-08,94.092682,94.169005,93.428664,93.863712,138019200
2010-12-09,94.459007,94.657446,93.993428,94.619288,123705100
2010-12-10,95.008545,95.100130,94.436112,94.749039,117571700


In [16]:

ffr = np.full(744, fed_funds_rate, dtype=float)

# 2. Convert daily SPY data to hourly array of length 744


spy_daily = data["Close"]

idx = pd.date_range("2010-12-01", "2010-12-31 23:00", freq="H")



/tmp/ipykernel_338171/4257181674.py:8: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range("2010-12-01", "2010-12-31 23:00", freq="H")


In [17]:
spy_daily

Ticker,SPY
Date,
2010-12-01,92.360092
2010-12-02,93.543129
2010-12-03,93.794998
2010-12-06,93.695793
2010-12-07,93.749237
2010-12-08,94.092682
2010-12-09,94.459007
2010-12-10,95.008545
2010-12-13,95.069626


We now have said data: rtm - (744), fed funds - (744), spy - (744), t2m - (744, 49, 58). I would like to predict the average real time value of electricity on the last day.

In [18]:
rtm_hourly.shape

(744, 1)

making it such that the weekend spy prices stay the same as the previous (market is closed on weekends)

In [19]:

spy_daily_full = spy_daily.asfreq("D").ffill()
idx = pd.date_range("2010-12-01", "2010-12-31", freq="D")
spy_daily_full = spy_daily.reindex(idx).ffill()

In [20]:
start = spy_daily_full.index.min()
end   = spy_daily_full.index.max()

idx_hourly = pd.date_range(start, end + pd.Timedelta(hours=23), freq="H")

spy_hourly = spy_daily_full.reindex(idx_hourly, method="ffill")
spy_hourly.shape

/tmp/ipykernel_338171/3564471548.py:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx_hourly = pd.date_range(start, end + pd.Timedelta(hours=23), freq="H")


(744, 1)

In [21]:
print(rtm_hourly.shape)
print(spy_hourly.shape)

print(temp_sub.shape)
print(ffr.shape)

(744, 1)
(744, 1)
(744, 49, 58)
(744,)


Having a Model which preserves Spatial Dimensions of T2M

In [22]:
import numpy as np
import pandas as pd

T = rtm_hourly.shape[0]  # 744 for one month
feature_cols = ["RT", "SPY", "FFR", "hour", "dow", "month"]

idx = pd.date_range("2010-12-01", periods=T, freq="H")  # adjust start

df_scalar = pd.DataFrame(index=idx)
df_scalar["RT"]   = rtm_hourly.squeeze()
df_scalar["SPY"]  = spy_hourly.squeeze()
df_scalar["FFR"]  = ffr.astype("float32")

# calendar features
df_scalar["hour"]  = df_scalar.index.hour
df_scalar["dow"]   = df_scalar.index.dayofweek
df_scalar["month"] = df_scalar.index.month


/tmp/ipykernel_338171/4271329016.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  idx = pd.date_range("2010-12-01", periods=T, freq="H")  # adjust start


In [23]:
mask_valid = ~df_scalar[feature_cols + ["RT"]].isna().any(axis=1)
df_scalar = df_scalar[mask_valid]
t2m_sub = temp_sub[mask_valid.to_numpy()]  # keep t2m in sync with time axis

In [24]:
df_scalar.head()

,RT,SPY,FFR,hour,dow,month
2010-12-01 01:00:00,23.4750,92.360092,0.18,1,2,12
2010-12-01 02:00:00,21.4675,92.360092,0.18,2,2,12
2010-12-01 03:00:00,21.3100,92.360092,0.18,3,2,12
2010-12-01 04:00:00,22.3500,92.360092,0.18,4,2,12
2010-12-01 05:00:00,24.3925,92.360092,0.18,5,2,12


In [25]:
H, W = temp_sub.shape[1], temp_sub.shape[2]

In [26]:
past_hours = 24
future_hours = 24



def make_supervised_spatiotemporal(df_scalar, t2m_grid, past_hours, future_hours, feature_cols):
    # df_scalar: DataFrame (T, n_scalar)
    # t2m_grid:  array (T, H, W)
    values = df_scalar[feature_cols].to_numpy().astype("float32")  # (T, n_scalar)
    rt     = df_scalar["RT"].to_numpy().astype("float32")          # (T,)
    
    T, n_scalar = values.shape
    H, W = t2m_grid.shape[1], t2m_grid.shape[2]
    
    X_grid_list = []
    X_seq_list  = []
    Y_list      = []
    
    for t in range(T - past_hours - future_hours + 1):
        # spatio-temporal temperature block
        grid_block = t2m_grid[t : t + past_hours]                 # (past_hours, H, W)
        grid_block = grid_block[..., np.newaxis]                  # (past_hours, H, W, 1)
        
        # scalar time-series block
        seq_block = values[t : t + past_hours]                    # (past_hours, n_scalar)
        
        # future RT prices
        future_rt = rt[t + past_hours : t + past_hours + future_hours]  # (24,)
        
        X_grid_list.append(grid_block)
        X_seq_list.append(seq_block)
        Y_list.append(future_rt)
    
    X_grid = np.stack(X_grid_list, axis=0)   # (n_samples, past_hours, H, W, 1)
    X_seq  = np.stack(X_seq_list, axis=0)    # (n_samples, past_hours, n_scalar)
    Y      = np.stack(Y_list, axis=0)        # (n_samples, future_hours)
    
    return X_grid, X_seq, Y

X_grid, X_seq, Y = make_supervised_spatiotemporal(
    df_scalar, temp_sub, past_hours, future_hours, feature_cols
)

print("X_grid:", X_grid.shape)  # (n_samples, 24, H, W, 1)
print("X_seq: ", X_seq.shape)   # (n_samples, 24, n_scalar)
print("Y:     ", Y.shape)       # (n_samples, 24)


X_grid: (696, 24, 49, 58, 1)
X_seq:  (696, 24, 6)
Y:      (696, 24)


In [27]:
from tensorflow.keras.layers import (
    Input, ConvLSTM2D, BatchNormalization, GlobalAveragePooling2D,
    Conv1D, GlobalAveragePooling1D, Dense, Concatenate
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

n_scalar = X_seq.shape[2]
H, W = X_grid.shape[2], X_grid.shape[3]

# ---- t2m spatio-temporal branch ----
grid_in = Input(shape=(past_hours, H, W, 1), name="grid_input")
xg = ConvLSTM2D(
    filters=16,
    kernel_size=(3, 3),
    padding="same",
    activation="relu",
    return_sequences=False
)(grid_in)
xg = BatchNormalization()(xg)
xg = GlobalAveragePooling2D()(xg)   # -> (batch, 16)

# ---- scalar time-series branch ----
seq_in = Input(shape=(past_hours, n_scalar), name="seq_input")
xs = Conv1D(filters=32, kernel_size=3, padding="causal", activation="relu")(seq_in)
xs = Conv1D(filters=32, kernel_size=3, padding="causal", activation="relu")(xs)
xs = GlobalAveragePooling1D()(xs)   # -> (batch, 32)

# ---- combine branches ----
x = Concatenate()([xg, xs])         # (batch, 48)
x = Dense(64, activation="relu")(x)
x = Dense(64, activation="relu")(x)
out = Dense(future_hours, name="output")(x)   # 24-hour forecast

model = Model(inputs=[grid_in, seq_in], outputs=out)
model.compile(optimizer=Adam(1e-3), loss="mse", metrics=["mae"])
model.summary()


2025-11-29 13:16:53.091264: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-29 13:16:53.468127: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-29 13:16:53.595389: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-29 13:16:53.632106: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-29 13:16:53.941457: I tensorflow/core/platform/cpu_feature_guar

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ grid_input          │ (None, 24, 49,    │          0 │ -                 │
│ (InputLayer)        │ 58, 1)            │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ seq_input           │ (None, 24, 6)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_lstm2d         │ (None, 49, 58,    │      9,856 │ grid_input[0][0]  │
│ (ConvLSTM2D)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d (Conv1D)     │ (None, 24, 32)    │        608 │ seq_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 49, 58,    │         64 │ conv_lstm2d[0][0] │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_1 (Conv1D)   │ (None, 24, 32)    │      3,104 │ conv1d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 16)        │          0 │ batch_normalizat… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 32)        │          0 │ conv1d_1[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 48)        │          0 │ global_average_p… │
│ (Concatenate)       │                   │            │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 64)        │      3,136 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 64)        │      4,160 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 24)        │      1,560 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 22,488 (87.84 KB)

 Trainable params: 22,456 (87.72 KB)

 Non-trainable params: 32 (128.00 B)

In [28]:
split = int(len(X_grid) * 0.8)
Xg_train, Xg_test = X_grid[:split], X_grid[split:]
Xs_train, Xs_test = X_seq[:split],  X_seq[split:]
Y_train,  Y_test  = Y[:split],      Y[split:]

history = model.fit(
    [Xg_train, Xs_train], Y_train,
    validation_data=([Xg_test, Xs_test], Y_test),
    epochs=10,
    batch_size=4,    # keep small if H,W still big
    verbose=1
)


Epoch 1/10


I0000 00:00:1764440222.476251  338376 service.cc:146] XLA service 0x7f34640184b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764440222.476367  338376 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2025-11-29 13:17:02.565271: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-29 13:17:02.935873: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2025-11-29 13:17:08.062370: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'input_multiply_reduce_select_fusion', 220 bytes spill stores, 220 bytes spill loads

I0000 00:00:1764440228.088153  338376 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the l

139/139 ━━━━━━━━━━━━━━━━━━━━ 17s 61ms/step - loss: 1403.6528 - mae: 16.9632 - val_loss: 93.0403 - val_mae: 7.6143
Epoch 2/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 1062.8951 - mae: 9.8832 - val_loss: 92.4546 - val_mae: 7.3906
Epoch 3/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 986.6799 - mae: 9.3588 - val_loss: 109.1647 - val_mae: 8.7839
Epoch 4/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step - loss: 1014.7551 - mae: 10.1585 - val_loss: 82.1779 - val_mae: 6.7253
Epoch 5/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 7s 53ms/step - loss: 940.9271 - mae: 9.3369 - val_loss: 67.6175 - val_mae: 5.4530
Epoch 6/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 8s 56ms/step - loss: 923.6083 - mae: 9.5785 - val_loss: 114.8509 - val_mae: 8.9817
Epoch 7/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 8s 55ms/step - loss: 948.4028 - mae: 9.8755 - val_loss: 83.9551 - val_mae: 6.9796
Epoch 8/10
139/139 ━━━━━━━━━━━━━━━━━━━━ 8s 58ms/step - loss: 1075.2518 - mae: 10.0675 - val_loss: 109.9351 - val_mae: 8.7593
Epoch 9/10
139/139 ━━━━━

In [29]:
last_grid_block = temp_sub[-past_hours:]              # (24, H, W)
last_grid_block = last_grid_block[..., np.newaxis]   # (24, H, W, 1)

last_seq_block = df_scalar[feature_cols].to_numpy().astype("float32")[-past_hours:]

Xg_last = np.expand_dims(last_grid_block, axis=0)    # (1, 24, H, W, 1)
Xs_last = np.expand_dims(last_seq_block,  axis=0)    # (1, 24, n_scalar)

next_24_forecast = model.predict([Xg_last, Xs_last])[0]  # shape (24,)
print("Next 24-hour forecast:", next_24_forecast)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Next 24-hour forecast: [30.18344  31.365366 30.85576  30.827059 35.47166  32.135254 31.603863
 30.11819  28.354145 30.590347 29.85586  31.651262 30.167412 28.654987
 31.993162 29.691158 29.358974 32.09681  30.730362 30.95743  30.85022
 36.327667 33.28593  32.569237]
